In [3]:
import os
import numpy as np
import pydicom
import zipfile
from tqdm import tqdm
from scipy.ndimage import zoom
import PIL.Image as Image
import cv2
import shutil

img_size = 512
slice_num = 128

def window_image(dcm, window_center, window_width, intercept, slope, rescale=True):
    img = dcm.pixel_array
    img = (img * slope + intercept)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max
    if rescale:
        img = (img - img_min) / (img_max - img_min)
        # img = (img * 255).astype(np.uint8)
    return img

def no_window_image(dcm, intercept, slope, rescale=True):
    img = dcm.pixel_array
    img = (img * slope + intercept)
    if rescale:
        img = (img - np.min(img)) / (np.max(img) - np.min(img))
        # img = (img * 255).astype(np.uint8)
    return img

def window_volume(volume, window_center, window_width, rescale=True):
    img = volume
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max
    if rescale:
        img = (img - img_min) / (img_max - img_min)
        # img = (img * 255).astype(np.uint8)
    return img

def normalize_image(image):
    image = image - np.min(image)
    return image / np.max(image)

def resize_volume(volume, target_shape=(slice_num, img_size, img_size)):
    original_shape = np.array(volume.shape)
    target_shape = np.array(target_shape)
    scale_factors = target_shape / original_shape
    resized_volume = zoom(volume, scale_factors, order=3)  # order=3 は三次元補間（三次スプライン補間）
    
    return resized_volume

def sort_dicoms_by_position(dicom_files):
    dicoms = [pydicom.dcmread(df, force=True) for df in dicom_files]
    dicoms.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    return dicoms

def create_npy_in_memory(dicom_files, window_center=None, window_width=None):
    dicoms = sort_dicoms_by_position(dicom_files)
    #volume = np.stack([no_window_image(dcm, dcm.RescaleIntercept, dcm.RescaleSlope, False) if window_center is not None and window_width is not None else dcm.pixel_array for dcm in dicoms])
    try:
        volume = np.stack([no_window_image(dcm, dcm.RescaleIntercept, dcm.RescaleSlope, False) if window_center is not None and window_width is not None else dcm.pixel_array for dcm in dicoms])
    except:
        volume = np.stack([cv2.resize(no_window_image(dcm, dcm.RescaleIntercept, dcm.RescaleSlope, False), (img_size, img_size)) if window_center is not None and window_width is not None else dcm.pixel_array for dcm in dicoms])
        print(dicom_files[0])
    for i, dcm in enumerate(dicoms):
        if dcm.PhotometricInterpretation == "MONOCHROME1":
            volume[i] = np.invert(volume[i])
    # volume = resize_volume(volume)
    volume_brain = window_volume(volume.copy(), 40, 80)
    volume_bone = window_volume(volume.copy(), 600, 2800)
    volume_soft = window_volume(volume.copy(), 60, 400)
    volume = np.stack([volume_brain, volume_bone, volume_soft]) * 255
    volume = volume.astype(np.uint8)
    return volume

def save_volume_to_zip(volume, accession_number, zip_file):
    with zip_file.open(accession_number + '.npy', 'w') as file_handle:
        np.save(file_handle, volume)
        
def save_slice_as_png_zip(volume, accession_number, zip_file):
    for i in range(volume.shape[1]):  # Loop over slices
        slice_img = volume[:,i,:,:]
        slice_img = slice_img.transpose(1,2,0)
        img = Image.fromarray(slice_img)
        img = img.convert("RGB")  # Convert to RGB for saving as PNG
        with zip_file.open(f"{accession_number}_slice_{i}.png", 'w') as file_handle:
            img.save(file_handle, format="PNG")

def save_slice_as_png(volume, accession_number):
    save_dir_tmp = os.path.join(output_base_dir, accession_number)
    os.makedirs(save_dir_tmp, exist_ok=True)
    for i in range(volume.shape[1]):  # Loop over slices
        slice_img = volume[:,i,:,:]
        slice_img = slice_img.transpose(1,2,0)
        slice_img = cv2.resize(slice_img, (img_size, img_size))
        save_path = os.path.join(save_dir_tmp, f"slice_{i}.png")
        cv2.imwrite(save_path, slice_img)

### convert train data to stacked png

In [ ]:
base_dir = "../data/dataset_jpr_train/dataset_jpr_train"
output_base_dir = f"../data/train_stacked_{img_size}"
zip_path = os.path.join(output_base_dir, 'npy_files.zip')
os.makedirs(output_base_dir, exist_ok=True)

roots = ["1", "2", "3"]
for root_dir in roots:
    root_path = os.path.join(base_dir, root_dir)
    for accession_number in tqdm(os.listdir(root_path)):
        accession_path = os.path.join(root_path, accession_number)
        dicom_files = [os.path.join(accession_path, f) for f in os.listdir(accession_path)]
        try:
            volume = create_npy_in_memory(dicom_files, window_center=40, window_width=80)
            save_slice_as_png(volume, accession_number)
        except Exception as e:
            print(f"ERROR processing volume in {accession_path}: {str(e)}")

### convert test data to stacked png

In [ ]:
base_dir = "../data/dataset_jpr_test2"
output_base_dir = f"../data/test_stacked_{img_size}" #"/kaggle/working/3d_numpy_files/"
os.makedirs(output_base_dir, exist_ok=True)
roots = ["dataset_jpr_test2"]
for root_dir in roots:
    root_path = os.path.join(base_dir, root_dir)
    for accession_number in tqdm(os.listdir(root_path)):
        accession_path = os.path.join(root_path, accession_number)
        dicom_files = [os.path.join(accession_path, f) for f in os.listdir(accession_path)]
        try:
            volume = create_npy_in_memory(dicom_files, window_center=40, window_width=80)
            # save_volume_to_zip(volume, accession_number, zip_file)
            save_slice_as_png(volume, accession_number)
        except Exception as e:
            print(f"ERROR processing volume in {accession_path}: {str(e)}")